In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np
import pandas as pd
import pickle

In [2]:
def get_style_links():
    # we start on the styles page because it is the easiest page 
    # from which to access all beers
    
    followable_links = []
    url = 'https://www.beeradvocate.com/beer/styles/'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    # get style links

    styles_links = [link for link in soup.find_all('a') if 'beer/styles' in str(link)]

    styles_nums = []

    for link in styles_links[2:]:
        styles_nums.append(str(link).split('/')[3])
        
    styles_nums = sorted(styles_nums)[1:]
    
    for num in styles_nums:
        followable_links.append(url + num)
    
    return followable_links

In [3]:
style_links = get_style_links()
style_links[0]

'https://www.beeradvocate.com/beer/styles/10'

In [84]:
def get_beer_info(url):
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    beer_info_td_s = soup.select('td[class*="hr_bottom_light"]')
    beer_info_list = [td.text for td in ratings_td]

    rearranger = int(len(beer_info_list)/6)

    beer_name = beer_info_list[::6]
    brewery_name = beer_info_list[1::6]
    abv = beer_info_list[2::6]
    ratings = beer_info_list[3::6]
    score = beer_info_list[4::6]

    beer_and_brewery_names_links = soup.select("a[href*=/beer/profile/]")

    brewery_nums = []

    for link in beer_and_brewery_names_links[1::2]:
        brew_num_string_list = str(link).split('/')
        brew_num = brew_num_string_list[3]
        brewery_nums.append(brew_num)

    data = np.array([beer_name,brewery_name,abv,
                     ratings,score,brewery_nums]).reshape(rearranger,6)

    df = pd.DataFrame(data=[beer_name,brewery_name,abv,
                     ratings,score,brewery_nums])

    df = df.transpose()

    df.columns = ['beer_name','brewery_name','abv',
                     'ratings','score','brewery_nums']
    
    return df.to_pickle()

In [86]:
df = get_beer_info('https://www.beeradvocate.com/beer/styles/10')

**psuedo code**

``` python


df = pd.DataFrame()

for each_link in style_links:
    
    page_of_beers = np.array()
    
    beer_names = call
    
    Brewery = call
    
    ABV = call
    
    Ratings = call
    
    score = call
    
    brewery_number = call
    
    page_of_beers = beer_names, brewery, ABV, ratings, score
    
    
    

In [5]:
def get_style_max(url):

    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    try:
        style_max_ref = soup.find_all('a')[-137]
        style_max = re.split(r'=|"', str(style_max_ref))[-2]
        style_max = int(style_max)
    except:
        style_max = 0
    
    return style_max

In [ ]:
def open_new_style_page(url,counter):
    out = url + '/?sort=revsD&start=' + str(counter)
    
    return out
    

In [ ]:
open_new_style_page(style_links[0],0)

# Master Function

In [ ]:
def compile_beer_info():
    
    style_links = get_style_links()
    
    style_links = [style_links[0]] # remove later
    
    for style_link in style_links:
        
        style_max = get_style_max(style_link)
        
        counter = 0

        while counter <= style_max:        

            url = open_new_style_page(style_link,counter)

            beer_info_df = get_beer_info(url)
            
            beer_info_df.to_pickle()